In [134]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import Select
import time

In [135]:
import pandas as pd
from datetime import datetime


pos = 0
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())




,login,senha,ubs
0,joana,05216791418,borges


In [136]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [137]:

def people(race_item):
    
    openEpHealph()
    print(f'Extração iniciada as: {datetime.now()}')
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")
    
    nav.find_element(By.XPATH, '//*[@id="img_diabetesFlag"]').click()
    time.sleep(1)
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    
    race.send_keys(Keys.ENTER)
    time.sleep(1)
    
    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()
    
    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(8)

In [138]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import traceback



def check_url():
    url = nav.current_url
#     print("The current url is:"+str(url))
    if url != "https://afogadosdaingazeira.ephealth.com.br/portal/individuo":
        nav.back()



def mothers_name():
    
    mothers = []
    exit = False
    second_click = 0

    
    people(3)
    time.sleep(3)
    
    page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    col = peoples.find_elements(By.TAG_NAME, 'td')
    link = peoples.find_elements(By.TAG_NAME, 'a')
    
    
    
    while exit is False:
        
        try:
            i = 1
            print(f'{i} - do try')
            
            while i < int(len(link)/2) + 1:
                print(f'{i} - do while inicio')
                check_url()

                time.sleep(8)
                nome = nav.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[2]/div[2]/div/div/div/div/div/div/div/div[3]/div[2]/table/tbody/tr[{i}]/td[2]/a")

                actions = ActionChains(nav)
                actions.key_down(Keys.CONTROL).perform()

        #         actions.move_to_element(nome).perform()
                nav.execute_script('arguments[0].scrollIntoView(true);', nome)
                time.sleep(2)
                actions.click(nome)
                actions.key_up(Keys.CONTROL).perform()

                nav.switch_to.window(nav.window_handles[1])
                time.sleep(3)
                mothers_name = nav.find_element(By.CSS_SELECTOR, 'div.card:nth-child(3) > div:nth-child(1) > div:nth-child(2) > table:nth-child(1) > tbody:nth-child(1) > tr:nth-child(6) > td:nth-child(2)')
                mothers.append(mothers_name.text)
                print(f'{i}-{mothers_name.text}')
                nav.close()
                nav.switch_to.window(nav.window_handles[0])
                
                i += 1
                print(f'{i} - do while final')
                
                if second_click >= 1:
                    check_url()
                    print(f'{i} - antes do IF')
                    
                    j = 0
                    for j in range(0, second_click):
                        time.sleep(6)
                        page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")
                        page.click()
                        print(f'{i} - depois do IF')
                        

                
                
            print("checou")
            check_url()
            print("passou")
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")
            print("prucurou")
            page.click()
            print("clicou")
            second_click += 1
            time.sleep(10) 
        
            
        except NoSuchElementException:
            print(NoSuchElementException)
            traceback.print_exc()
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            col = peoples.find_elements(By.TAG_NAME, 'td')
            link = peoples.find_elements(By.TAG_NAME, 'a')
        
        """except ElementClickInterceptedException:
            check_url()
            time.sleep(3)
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")
            page.click()"""

    
    
    return mothers

mamae = mothers_name()



Extração iniciada as: 2024-01-21 21:00:18.430528
1 - do try
1 - do while inicio
1-APARECIDA MACHADO DA COSTA
2 - do while final
2 - do while inicio
2-MARIA DO SOCORRO DE ARAUJO
3 - do while final
3 - do while inicio
3-MARIA DE FATIMA GOMES DE QUEIROZ
4 - do while final
4 - do while inicio
4-ALZIRA BEZERRA DOS RAMOS
5 - do while final
5 - do while inicio
5-JUDITE SABINO GOMES
6 - do while final
6 - do while inicio
6-MARIA DE LURDES LUCAS DA SILVA
7 - do while final
7 - do while inicio
7-ALICE MORATO BARBOZA
8 - do while final
8 - do while inicio
8-JOSEFA DE CARVALHO BARROS
9 - do while final
9 - do while inicio
9-FRANCISCA ANA DA SILVA
10 - do while final
10 - do while inicio
10-TERESA XAVIER DE LIMA
11 - do while final
11 - do while inicio
11-DALVA BARBOSA DA SILVA
12 - do while final
12 - do while inicio
12-NÚBIA REJANE VIEIRA COSTA
13 - do while final
13 - do while inicio
13-ESMERALDA DA SILVA
14 - do while final
14 - do while inicio
14-MARIA JOSÉ TEIXEIRA
15 - do while final
15 - do

47 - depois do IF
47 - do while inicio
47-MARIA PASTORA DA CONCEIÇÃO
48 - do while final
48 - antes do IF
48 - depois do IF
48 - do while inicio
48-MARIA DO CARMO NUNES DE AZEVEDO
49 - do while final
49 - antes do IF
49 - depois do IF
49 - do while inicio
49-MARIA OLINDRINA COSTA
50 - do while final
50 - antes do IF
50 - depois do IF
50 - do while inicio
50-QUITERIA FERREIRA DE OLIVEIRA
51 - do while final
51 - antes do IF
51 - depois do IF
checou
passou
prucurou
clicou
1 - do try
1 - do while inicio
1-MARIA DE LOURDES SIQUEIRA SILVA
2 - do while final
2 - antes do IF
2 - depois do IF
2 - depois do IF
2 - do while inicio
2-ANTONIA MARIA DA CONCEICAO
3 - do while final
3 - antes do IF
3 - depois do IF
3 - depois do IF
3 - do while inicio
3-MARIA SANTANA DA SILVA SIMÃO
4 - do while final
4 - antes do IF
4 - depois do IF
4 - depois do IF
4 - do while inicio
4-TERESINHA SIQUEIRA FERREIRA
5 - do while final
5 - antes do IF
5 - depois do IF
5 - depois do IF
5 - do while inicio
5-MARIA DA GLO

Traceback (most recent call last):
  File "/tmp/ipykernel_731763/1778649503.py", line 44, in mothers_name
    nome = nav.find_element(By.XPATH, f"/html/body/div[1]/div[2]/div/div[2]/div[2]/div/div/div/div/div/div/div/div[3]/div[2]/table/tbody/tr[{i}]/td[2]/a")
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/webdriver.py", line 976, in find_element
    return self.execute(Command.FIND_ELEMENT, {
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/home/ricar/anaconda3/lib/python3.10/site-packages/selenium/webdriver/remote/errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Message: Unable to locate element: /html/body/div[1]/div[2]/div/div[2]/div[2]/div/div/div/div/div/div/div/div[3]/div[2]/table/tbody/tr[36]/td[2]/a



In [145]:
print(len(mamae))

135


In [139]:
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def get_table(race_item):
    
    table = []
    people(race_item)
    
    exit = False
    
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    col = peoples.find_elements(By.TAG_NAME, 'td')
    
    while exit is False:
        
        try:            
            
            for i in col:
                table.append(i.text)
            
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
            """if ElementClickInterceptedException:    
                print(ElementClickInterceptedException.with_traceback)
                
            if NoSuchElementException: 
                print(NoSuchElementException.with_traceback)"""
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            col = peoples.find_elements(By.TAG_NAME, 'td')
    
    nav.quit()
    print(f'Extração finalizada as: {datetime.now()}')
    return table


In [140]:
def build_df(table, race_item):
    '''name cns cpf age gender last_visit profissional team unity'''

    df_people = pd.DataFrame()

    name = []
    cns = []
    cpf = []
    age = []
    gender = []
    last_visit = []
    profissional = []
    team = []
    unity = []
    race_col = []


    for nam in range(1, len(table), 10):
        name.append(table[nam])

    for cn in range(2, len(table), 10):
        cns.append(table[cn])

    for cp in range(3, len(table), 10):
        cpf.append(table[cp])

    for ag in range(4, len(table), 10):
        age.append(table[ag])

    for gen in range(5, len(table), 10):
        gender.append(table[gen])

    for lv in range(6, len(table), 10):
        last_visit.append(table[lv])

    for prof in range(7, len(table), 10):
        profissional.append(table[prof])

    for t in range(8, len(table), 10):
        team.append(table[t])

    for unt in range(9, len(table), 10):
        unity.append(table[unt])
    if table[0] != 'Nenhum indivíduo encontrado':
        for race in range(0, len(table), 10):
            race_col.append(raceList[race_item])

    df_people['name'] = name
    df_people['cns'] = cns
    df_people['cpf'] = cpf
    df_people['age'] = age
    df_people['age'] = df_people['age'].astype(str).str[4:-1]

    df_people['gender'] = gender
    df_people['last_visit'] = last_visit
    df_people['profissional'] = profissional
    df_people['team'] = team
    df_people['unity'] = unity
    df_people['race_col'] = race_col

#     display(df_people)
    
#     df_people.to_excel('tabela.xlsx', index=False)
    
    return df_people


In [141]:
def complet_extraction():
    
    j = 0
    complete_df = pd.DataFrame()
    pre_df = pd.DataFrame()
    pre_list = []
    
    for i in range(0, len(raceList)):
        pre_list = get_table(j)
        pre_df = build_df(pre_list, j)

        complete_df = pd.concat([complete_df, pre_df], ignore_index= True)
    
    display(complete_df)
    print(f'Extração finalizada as: {datetime.now()}')
    return complete_df
#         complete_df = pd.concat([complete_df, suap_df], ignore_index= True)




In [142]:
# exibi = complet_extraction()
# print(type(exibi))
# print(exibi.info())
# display(exibi)
# exibi.to_excel(f'diabetic/{data_e_hora}_{autentica["ubs"][0]}_diabetic.xlsx', index=False)
# people(2)

# clear_race_field()
# jeba = get_table(0)

In [143]:
# build_df(jeba, 0)